In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time

In [5]:
land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8706295, 11)


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 100-0,2013,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,325000.0
1,서울특별시 강남구 개포동 100-0,2014,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,330000.0
2,서울특별시 강남구 개포동 100-0,2015,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,335000.0
3,서울특별시 강남구 개포동 100-0,2016,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,345000.0
4,서울특별시 강남구 개포동 100-0,2017,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,355000.0


In [6]:
last_df = land_specs_df.drop_duplicates(subset=['지번주소'], keep='last').drop(columns=['년', '공시지가']).reset_index(drop=True)
print(last_df.shape)
last_df.head()

(981464, 9)


,지번주소,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 100-0,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
1,서울특별시 강남구 개포동 101-0,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지
2,서울특별시 강남구 개포동 102-0,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지
3,서울특별시 강남구 개포동 103-0,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지
4,서울특별시 강남구 개포동 104-1,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지


In [2]:
def get_gu_dong(df):
    df = df.copy()
    
    df['구'] = df['지번주소'].apply(lambda x: x.split(' ')[1])
    df['동'] = df['지번주소'].apply(lambda x: x.split(' ')[2])
    
    return df

In [3]:
bulktrade_basedir = './국토교통부_실거래가_공개시스템/집값분석/일괄구매/10호이상/'

In [4]:
date_today = pd.to_datetime(datetime.datetime.now())
target_period = 6 # in months

In [9]:
yunrip_bulk_df = pd.read_csv(bulktrade_basedir+'연립다세대_일괄구매_전체.csv')
yunrip_bulk_df['계약날짜'] = pd.to_datetime(yunrip_bulk_df['계약날짜'])
#yunrip_bulk_df = yunrip_bulk_df[yunrip_bulk_df['계약날짜']  >= (date_today - pd.DateOffset(months=target_period))]
yunrip_bulk_df = yunrip_bulk_df[
    (yunrip_bulk_df['소유자'] == '서***~')
    |(yunrip_bulk_df['소유자'] == '한***~')
].reset_index(drop=True)
yunrip_bulk_df = get_gu_dong(yunrip_bulk_df)
print(yunrip_bulk_df.shape)
yunrip_bulk_df.head()

(15596, 16)


,지번주소,도로명,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동
0,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,11.38,25050,2,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동
1,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,13.16,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동
2,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,13.16,28950,3,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동
3,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,11.38,25050,3,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동
4,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,13.16,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동


In [10]:
officetel_bulk_df = pd.read_csv(bulktrade_basedir+'오피스텔_일괄구매_전체.csv')
officetel_bulk_df['계약날짜'] = pd.to_datetime(officetel_bulk_df['계약날짜'])
#officetel_bulk_df = officetel_bulk_df[officetel_bulk_df['계약날짜']  >= (date_today - pd.DateOffset(months=target_period))]
officetel_bulk_df = officetel_bulk_df[
    (officetel_bulk_df['소유자'] == '서***~')
    |(officetel_bulk_df['소유자'] == '한***~')
].reset_index(drop=True)
officetel_bulk_df = get_gu_dong(officetel_bulk_df)
print(officetel_bulk_df.shape)
officetel_bulk_df.head()

(1254, 15)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동
0,서울특별시 금천구 독산동 332-13,범안로15길 21,명도레이크캐슬,17.12,13150,2,2019.0,0.0,2019-08-16,768.107477,2.0,오피스텔,한***~,금천구,독산동
1,서울특별시 금천구 독산동 332-13,범안로15길 21,명도레이크캐슬,17.12,13150,2,2019.0,0.0,2019-08-16,768.107477,2.0,오피스텔,한***~,금천구,독산동
2,서울특별시 금천구 독산동 332-13,범안로15길 21,명도레이크캐슬,17.12,13550,2,2019.0,0.0,2019-08-16,791.471963,2.0,오피스텔,한***~,금천구,독산동
3,서울특별시 금천구 독산동 332-13,범안로15길 21,명도레이크캐슬,17.12,13550,2,2019.0,0.0,2019-08-16,791.471963,2.0,오피스텔,한***~,금천구,독산동
4,서울특별시 금천구 독산동 332-13,범안로15길 21,명도레이크캐슬,17.12,13550,2,2019.0,0.0,2019-08-16,791.471963,2.0,오피스텔,한***~,금천구,독산동


In [19]:
print(yunrip_bulk_df.shape)
print(officetel_bulk_df.shape)

(15596, 17)
(1254, 16)


In [20]:
yunrip_bulk_df = yunrip_bulk_df.drop(columns=['대지권면적(㎡)'])

In [21]:
concat_df = pd.concat([yunrip_bulk_df, officetel_bulk_df]).reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

(16850, 16)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동,구별_최대단가
0,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,25050,2,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동,1053.167133
1,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,1053.167133
2,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,3,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,1053.167133
3,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,25050,3,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동,1053.167133
4,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,1053.167133


In [22]:
sales_idx_basedir = './한국부동산원/실거래가격지수/'
filenames = [f for f in os.listdir(sales_idx_basedir) if '연립다세대' in f]

sales_idx_df = pd.read_excel(sales_idx_basedir + filenames[0], header=[10, 11])

sales_idx_cols_list = []
for i in range(sales_idx_df.shape[1]):
    if i == 0:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0])
    else:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0] + '_' + sales_idx_df.columns[i][1])

sales_idx_df.columns = sales_idx_cols_list

In [23]:
def get_sales_idx(x):
    # x is a pandas timestamp
    
    df = sales_idx_df.copy()
    
    if pd.isna(x) == True:
        return x
    else:
        year = str(x.year)
        month = str(x.month)
        if len(month) == 1:
            month = '0'+month
        
        idx = df[year+'년 '+month+'월_지수'].iloc[0]
        
        return idx

In [24]:
def get_index_applied_prices(df):
    df = df.copy()
    
    last_idx = sales_idx_df.iloc[0,-2]
    
    last_date = sales_idx_df.columns[-1].split('_')[0]
    
    last_date = last_date.split(' ')[0].replace('년','') + '-' + last_date.split(' ')[1].replace('월','') + '-01'
    
    target_upper_limit_df = df[df['계약날짜'] < pd.to_datetime(last_date)]
    target_upper_limit_df['계약날짜_실거래지수'] = target_upper_limit_df['계약날짜'].apply(get_sales_idx)
    target_upper_limit_df['지수적용단가'] = target_upper_limit_df['단가(만원/㎡)'] / target_upper_limit_df['계약날짜_실거래지수'] * last_idx
    df['지수적용단가'] = df['단가(만원/㎡)']
    df.loc[target_upper_limit_df.index, '지수적용단가'] = target_upper_limit_df['지수적용단가']
    df['평가날짜'] = df['계약날짜']
    df.loc[target_upper_limit_df.index, '평가날짜'] = pd.to_datetime(last_date[:-2] + '01')
    
    return df

In [25]:
concat_df = get_index_applied_prices(concat_df)
print(concat_df.shape)
concat_df.head()

(16850, 18)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동,구별_최대단가,지수적용단가,평가날짜
0,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,25050,2,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동,1053.167133,867.330392,2021-03-01
1,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,1053.167133,867.455335,2021-03-01
2,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,3,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,1053.167133,867.455335,2021-03-01
3,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,25050,3,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동,1053.167133,867.330392,2021-03-01
4,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,1053.167133,867.455335,2021-03-01


In [28]:
concat_df['구별_최대단가'] = concat_df[['구', '지수적용단가']].groupby(['구']).transform('max')['지수적용단가']
concat_df.head()

,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동,지수적용단가,평가날짜,구별_최대단가
0,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,25050,2,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동,867.330392,2021-03-01,1093.775181
1,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,867.455335,2021-03-01,1093.775181
2,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,3,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,867.455335,2021-03-01,1093.775181
3,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,39.93,25050,3,2015.0,0.0,2015-11-17,627.347859,6.0,연립다세대,한***~,강동구,길동,867.330392,2021-03-01,1093.775181
4,서울특별시 강동구 길동 376-2,천중로40길 27,진성파크빌,46.14,28950,2,2015.0,0.0,2015-11-17,627.438231,6.0,연립다세대,한***~,강동구,길동,867.455335,2021-03-01,1093.775181


In [33]:
gu_max_idx = []
for gu in concat_df['구'].unique():
    gu_df = concat_df[concat_df['구'] == gu]
    
    max_df = gu_df[gu_df['지수적용단가'] == gu_df['구별_최대단가'].iloc[0]]
    
    [gu_max_idx.append(i) for i in max_df.index]

In [34]:
gu_max_df = concat_df.loc[gu_max_idx]
print(gu_max_df.shape)
gu_max_df.head()

(66, 18)


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동,지수적용단가,평가날짜,구별_최대단가
9829,서울특별시 강동구 성내동 461-3,성내로6다길 55,서도휴빌,44.13,42300,4,2019.0,1.0,2020-01-16,958.531611,2.0,연립다세대,한***~,강동구,성내동,1093.775181,2021-03-01,1093.775181
6363,서울특별시 강북구 미아동 164-2,덕릉로30길 49,청운美家,30.00,23350,4,2019.0,0.0,2019-11-08,778.333333,2.0,연립다세대,한***~,강북구,미아동,874.069274,2021-03-01,874.069274
6367,서울특별시 강북구 미아동 164-2,덕릉로30길 49,청운美家,30.00,23350,3,2019.0,0.0,2019-11-08,778.333333,2.0,연립다세대,한***~,강북구,미아동,874.069274,2021-03-01,874.069274
6373,서울특별시 강북구 미아동 164-2,덕릉로30길 49,청운美家,30.00,23350,3,2019.0,0.0,2019-11-08,778.333333,2.0,연립다세대,한***~,강북구,미아동,874.069274,2021-03-01,874.069274
6389,서울특별시 강북구 미아동 164-2,덕릉로30길 49,청운美家,30.00,23350,5,2019.0,0.0,2019-11-08,778.333333,2.0,연립다세대,한***~,강북구,미아동,874.069274,2021-03-01,874.069274


In [35]:
gu_max_df = gu_max_df.drop(columns=['도로명', '건물(단지)명', '거래금액(만원)', '층', '계약날짜기준_건물연식',
                                    '계약날짜', '단가(만원/㎡)', '건물연식', '지수적용단가', '평가날짜'
                                   ]).drop_duplicates().reset_index(drop=True)
print(gu_max_df.shape)
gu_max_df.head()

(24, 8)


,지번주소,전용면적(㎡),건축년도,부동산유형,소유자,구,동,구별_최대단가
0,서울특별시 강동구 성내동 461-3,44.13,2019.0,연립다세대,한***~,강동구,성내동,1093.775181
1,서울특별시 강북구 미아동 164-2,30.00,2019.0,연립다세대,한***~,강북구,미아동,874.069274
2,서울특별시 강서구 화곡동 1051-24,14.09,2020.0,연립다세대,한***~,강서구,화곡동,1507.000459
3,서울특별시 구로구 개봉동 130-32,29.77,2016.0,연립다세대,한***~,구로구,개봉동,1330.852568
4,서울특별시 노원구 공릉동 561-3,22.81,2019.0,연립다세대,한***~,노원구,공릉동,980.485478


In [36]:
gu_max_df

,지번주소,전용면적(㎡),건축년도,부동산유형,소유자,구,동,구별_최대단가
0,서울특별시 강동구 성내동 461-3,44.13,2019.0,연립다세대,한***~,강동구,성내동,1093.775181
1,서울특별시 강북구 미아동 164-2,30.00,2019.0,연립다세대,한***~,강북구,미아동,874.069274
2,서울특별시 강서구 화곡동 1051-24,14.09,2020.0,연립다세대,한***~,강서구,화곡동,1507.000459
3,서울특별시 구로구 개봉동 130-32,29.77,2016.0,연립다세대,한***~,구로구,개봉동,1330.852568
4,서울특별시 노원구 공릉동 561-3,22.81,2019.0,연립다세대,한***~,노원구,공릉동,980.485478
5,서울특별시 도봉구 창동 657-11,23.41,2020.0,연립다세대,한***~,도봉구,창동,845.792396
6,서울특별시 서대문구 대현동 90-26,19.25,2019.0,오피스텔,한***~,서대문구,대현동,1209.875008
7,서울특별시 성북구 안암동1가 317-14,44.52,2018.0,연립다세대,한***~,성북구,안암동1가,1022.277326
8,서울특별시 송파구 송파동 90-4,19.58,2019.0,연립다세대,한***~,송파구,송파동,1719.705147
9,서울특별시 은평구 역촌동 44-2,16.12,2020.0,연립다세대,한***~,은평구,역촌동,1406.889244


In [38]:
gu_max_df['구'].nunique()

24

In [40]:
gu_max_df.to_excel('구별_최대단가.xlsx', index=False)

In [ ]:
concat_df

In [ ]:
yunrip_selected_df = yunrip_bulk_df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '지번별_전용면적별_평균단가', '부동산유형', '소유자']].drop_duplicates().reset_index(drop=True)
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df = yunrip_selected_df.sort_values(['지번주소']).reset_index(drop=True)
yunrip_selected_df.to_excel(bulktrade_basedir + '최근_6개월_SH_LH_거래사례.xlsx', index=False)

In [ ]:
yunrip_selected_df['지번별_평균단가'] = yunrip_selected_df[['지번주소', '건축년도', '계약날짜', '지번별_전용면적별_평균단가']].groupby(['지번주소', '건축년도', '계약날짜']).transform('mean')['지번별_전용면적별_평균단가']

In [ ]:
yunrip_selected_df = yunrip_selected_df.drop(columns=['전용면적(㎡)', '지번별_전용면적별_평균단가']).drop_duplicates().reset_index(drop=True)
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df['구'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[1])
yunrip_selected_df['동'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[2])
yunrip_selected_df['구동'] = yunrip_selected_df['구'] + '_' + yunrip_selected_df['동']

In [ ]:
basedir = './국토교통부_실거래가_공개시스템/땅값분석/'
yunrip_df = pd.read_csv(basedir + 'yunrip_25.csv')
commerce_df = pd.read_csv(basedir + 'commerce_25.csv')
land_df = pd.read_csv(basedir + 'land.csv')

In [ ]:
common_cols = ['시군구', '번지', '용도지역', '도로조건', '계약날짜', '대지면적(㎡)', '거래금액(만원)', '단가(㎡)', '단가(평)', '부동산유형', '건물연식']
common_df = pd.concat([yunrip_df[common_cols], commerce_df[common_cols], land_df[[col for col in land_df.columns if col not in ['지목', '지분구분', '해제사유발생일']]]])
common_df['계약날짜'] = pd.to_datetime(common_df['계약날짜'], format='%Y-%m-%d')
print(common_df.shape)
common_df.head()

In [ ]:
common_df['구'] = common_df['시군구'].apply(lambda x: x.split(' ')[1])
common_df['동'] = common_df['시군구'].apply(lambda x: x.split(' ')[2])

In [ ]:
yunrip_bulk_df['구동'] = yunrip_bulk_df['구'] + '_' + yunrip_bulk_df['동']

In [ ]:
summary_df = pd.DataFrame({'지역':[], '2종일반중간값':[], '집값중간값':[]})

count = 0
for gudong in tqdm(yunrip_selected_df['구동'].unique()):
    gu = gudong.split('_')[0]
    dong = gudong.split('_')[1]
    
    target_gu = gu
    target_dong = dong
    target_purpose = '2종일반'
    
    target_df = common_df[
        (common_df['구'] == target_gu)
        &(common_df['동'] == target_dong)
        &(common_df['용도지역'].str.contains(target_purpose))
        &(common_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
        &(common_df['단가(㎡)'] > 250)
    ].sort_values(['번지'])
    
    yunrip_target_df = yunrip_selected_df[
        (yunrip_selected_df['구동'] == gudong)
    ]
    
    summary_df.loc[count, '지역'] = gudong
    summary_df.loc[count, '2종일반중간값'] = target_df['단가(㎡)'].median()
    summary_df.loc[count, '집값중간값'] = yunrip_target_df['지번별_평균단가'].median()
    
    count += 1

In [ ]:
summary_df['차이'] = summary_df['집값중간값'] - summary_df['2종일반중간값']

In [ ]:
summary_df = summary_df.sort_values(['차이'], ascending=False).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df

In [ ]:
summary_df.to_excel('지역별_땅값_일괄매매_중간값.xlsx', index=False)

In [ ]:
summary_df = pd.DataFrame({'지역':[], '2종일반평균':[], '집값평균':[]})

count = 0
for gudong in tqdm(yunrip_bulk_df['구동'].unique()):
    gu = gudong.split('_')[0]
    dong = gudong.split('_')[1]
    
    target_gu = gu
    target_dong = dong
    target_purpose = '2종일반'
    
    target_df = common_df[
        (common_df['구'] == target_gu)
        &(common_df['동'] == target_dong)
        &(common_df['용도지역'].str.contains(target_purpose))
        &(common_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
        &(common_df['단가(㎡)'] > 250)
    ].sort_values(['번지'])
    
    yunrip_target_df = yunrip_selected_df[
        (yunrip_selected_df['구동'] == gudong)
    ]
    
    summary_df.loc[count, '지역'] = gudong
    summary_df.loc[count, '2종일반평균'] = target_df['단가(㎡)'].mean()
    summary_df.loc[count, '집값평균'] = yunrip_target_df['지번별_평균단가'].mean()
    
    count += 1

In [ ]:
summary_df['차이'] = summary_df['집값평균'] - summary_df['2종일반평균']

In [ ]:
summary_df = summary_df.sort_values(['차이'], ascending=False).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df

In [ ]:
summary_df.to_excel('지역별_땅값_일괄매매_평균.xlsx', index=False)

In [ ]:
yunrip_selected_df.head()

In [ ]:
inv_df = yunrip_selected_df[yunrip_selected_df['구동'] == '성동구_마장동']
print(inv_df.shape)
inv_df.head()

In [ ]:
def mean_prices_per_size(df):
    df = df.copy()
    
    df['크기별_평균단가'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['단가(만원/㎡)']
    df['크기별_평균총액'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '거래금액(만원)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['거래금액(만원)']
    df = df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '크기별_평균총액', '크기별_평균단가']]
    df = df.drop_duplicates().reset_index(drop=True)
    
    return df

In [ ]:
yunrip_short_df = mean_prices_per_size(yunrip_selected_df)
print(yunrip_short_df.shape)
yunrip_short_df.head()